In [1]:
import pandas as pd
import os
import requests
import json
import time
import ollama

In [ ]:
# --- 1. Definição dos Caminhos e Nomes de Colunas ---

# Caminho para a pasta onde o arquivo está localizado
caminho_pasta = '../data/raw'

# Nome do arquivo CSV
nome_arquivo = 'Cópia de Histeroscopia Diagnóstica (respostas) 2023 - Respostas ao formulário 1.csv'

# Construção do caminho completo do arquivo
caminho_completo = os.path.join(caminho_pasta, nome_arquivo)

# Lista com o nome exato das colunas que devem ser carregadas
colunas_para_carregar = [
    "Idade",
    "Indique sua condição de saúde atual",
    "Conte um pouco sobre o motivo para solicitar a consulta. ",
    "Número de prontuário no CISAM\nVocê só poderá continuar este preenchimento se tiver prontuário, caso vc não tenha prontuário no CISAM, clique aqui."
]

# --- 2. Carregamento do Arquivo CSV ---

try:
    # Carrega o arquivo CSV especificando apenas as colunas desejadas
    df = pd.read_csv(caminho_completo, usecols=colunas_para_carregar)

    # --- NOVA SEÇÃO: 3. Renomeando as Colunas ---

    # Dicionário para mapear os nomes antigos para os novos
    # 'nome antigo': 'nome novo'
    novos_nomes = {
        "Idade": "idade",
        "Indique sua condição de saúde atual": "condicao_saude",
        "Conte um pouco sobre o motivo para solicitar a consulta. ": "motivo_consulta",
        "Número de prontuário no CISAM\nVocê só poderá continuar este preenchimento se tiver prontuário, caso vc não tenha prontuário no CISAM, clique aqui.": "prontuario_cisam"
    }

    # A função rename é chamada no dataframe 'df'.
    # O parâmetro 'columns' recebe o dicionário com os novos nomes.
    # 'inplace=True' modifica o dataframe diretamente, sem precisar criar um novo.
    df.rename(columns=novos_nomes, inplace=True)

    # --- 4. Exibição dos Dados com as Colunas Renomeadas ---

    print("Dados carregados e colunas renomeadas com sucesso!")
    print("Visualizando as 5 primeiras linhas com os novos nomes de colunas:")
    display(df.head())

    print("\nInformações sobre o DataFrame (note os novos nomes nas colunas):")
    df.info()

except FileNotFoundError:
    print(f"Erro: O arquivo não foi encontrado no caminho especificado:\n{caminho_completo}")
    print("\nPor favor, verifique se o nome do arquivo e o caminho para a pasta '../data/raw' estão corretos.")
except ValueError as e:
    print(f"Erro ao carregar as colunas. Verifique se os nomes das colunas em 'colunas_para_carregar' estão exatamente como no arquivo CSV.")
    print(f"Detalhe do erro: {e}")

In [ ]:
# --- Garante que o DataFrame 'df' existe antes de iniciar ---
if 'df' in locals():

    # --- 0. Ponto de Partida: Análise inicial ---
    print("--- ANÁLISE INICIAL DO DATAFRAME 'df' ---")
    print(f"Número total de linhas (registros) antes da operação: {len(df)}")
    print(f"Número de prontuários únicos: {df['prontuario_cisam'].nunique()}")
    print("-" * 50)


    # --- 1. Remover todos os dados nulos da coluna 'prontuario_cisam' ---
    print("\n--- ETAPA 1: Removendo registros com prontuário nulo/vazio ---")
    df_clean = df.dropna(subset=['prontuario_cisam']).copy()

    # Garantia extra: remove também strings que contêm apenas espaços em branco
    # .astype(str) previne erros caso a coluna tenha tipos mistos
    df_clean = df_clean[df_clean['prontuario_cisam'].astype(str).str.strip() != '']
    print(f"Número de linhas após remover nulos: {len(df_clean)}")
    print("-" * 50)


    # --- 2. Unir valores repetidos com regras específicas ---
    print("\n--- ETAPA 2: Consolidando prontuários duplicados ---")
    print("Agrupando por 'prontuario_cisam' e aplicando regras de união...")

    # Define as regras de agregação para cada coluna
    regras_de_agregacao = {
        'idade': 'first',  # Pega o primeiro valor encontrado para a idade
        'condicao_saude': lambda s: ' | '.join(s.dropna().unique()), # Concatena valores únicos com ' | '
        'motivo_consulta': lambda s: ' | '.join(s.dropna().unique()) # Concatena valores únicos com ' | '
    }

    # Agrupa pelo prontuário e aplica as regras definidas
    # O .reset_index() transforma a coluna de agrupamento ('prontuario_cisam') de volta em uma coluna normal
    df_consolidado = df_clean.groupby('prontuario_cisam').agg(regras_de_agregacao).reset_index()

    print(f"Número de linhas após consolidar duplicados: {len(df_consolidado)}")
    print("Este número deve ser igual ao de prontuários únicos da análise inicial.")
    print("-" * 50)


    # --- 3. Verificar se sobraram valores repetidos ---
    print("\n--- ETAPA 3: Verificação final de duplicados ---")

    duplicados_restantes = df_consolidado['prontuario_cisam'].duplicated().any()

    if not duplicados_restantes:
        print("✅ SUCESSO! Não há mais nenhum valor duplicado na coluna 'prontuario_cisam'.")
    else:
        print("❌ ATENÇÃO! A operação falhou e ainda existem prontuários duplicados.")

    print("-" * 50)


    # --- 4. Exibição do Resultado Final ---
    print("\n--- RESULTADO FINAL ---")
    print("Visualizando as 5 primeiras linhas do DataFrame consolidado e limpo:")
    display(df_consolidado.head())

    # Verificando um exemplo de prontuário que pode ter sido concatenado (se houver duplicados)
    # print("\nExemplo de um registro que pode ter sido consolidado:")
    # display(df_consolidado[df_consolidado['motivo_consulta'].str.contains('\|', na=False)])


else:
    print("Erro: O DataFrame 'df' não foi encontrado. Por favor, execute a célula que o carrega primeiro.")

In [ ]:
# --- 1. Definição do Caminho e Colunas ---

# Nome do arquivo CSV
nome_arquivo_pacientes = 'Dados pacientes - Página1.csv'

# Construção do caminho completo para o arquivo
caminho_completo_pacientes = os.path.join(caminho_pasta, nome_arquivo_pacientes)

# --- Lista com as colunas específicas para carregar ---
colunas_pacientes_para_carregar = [
    "Prontuário",
    "Evolução"
]

# Dicionário para especificar que a coluna 'Prontuário' deve ser lida como texto (string)
tipos_dados = {
    'Prontuário': str
}


# --- 2. Carregamento do Arquivo CSV ---

try:
    # MODIFICADO: Adicionado o parâmetro 'dtype' para forçar o tipo da coluna
    df_pacientes = pd.read_csv(
        caminho_completo_pacientes, 
        usecols=colunas_pacientes_para_carregar,
        dtype=tipos_dados
    )

    # --- 3. Exibição dos Dados Carregados ---

    print("Arquivo de pacientes carregado com sucesso (apenas colunas selecionadas)!")
    print("Visualizando as 5 primeiras linhas:")
    display(df_pacientes.head())

    print("\nInformações sobre o DataFrame de pacientes (note as colunas carregadas):")
    # Agora, a saída de .info() mostrará 'Prontuário' como 'object' ou 'string', que é o tipo para texto no pandas.
    df_pacientes.info()

except FileNotFoundError:
    print(f"Erro: O arquivo não foi encontrado no caminho especificado:\n{caminho_completo_pacientes}")
    print("\nPor favor, verifique se o nome do arquivo está correto e se ele se encontra na pasta '../data/raw'.")
except ValueError as e:
    print(f"Erro ao carregar as colunas. Verifique se os nomes 'Prontuário' e 'Evolução' estão exatamente como no arquivo CSV.")
    print(f"Detalhe do erro: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao tentar carregar o arquivo: {e}")

In [ ]:
# --- Garante que o DataFrame 'df_pacientes' existe antes de iniciar ---
if 'df_pacientes' in locals():

    print("--- Operando no DataFrame 'df_pacientes' ---")
    print(f"Número de linhas ANTES da limpeza: {len(df_pacientes)}")

    # --- 1. Remover valores vazios na coluna 'Prontuário' ---

    # Criamos uma cópia para não alterar o DataFrame original
    df_pacientes_limpo = df_pacientes.copy()

    # Etapa 1.1: Remove valores nulos (NaN)
    df_pacientes_limpo.dropna(subset=['Prontuário'], inplace=True)

    # Etapa 1.2: Como a coluna é String, removemos também strings vazias ou que contêm apenas espaços
    # O .astype(str) garante que o método .str funcione sem erros
    # O .str.strip() remove espaços em branco do início e do fim
    df_pacientes_limpo = df_pacientes_limpo[df_pacientes_limpo['Prontuário'].astype(str).str.strip() != '']

    print(f"Número de linhas APÓS a limpeza: {len(df_pacientes_limpo)}")
    print("-" * 50)


    # --- 2. Verificar se existem valores duplicados após a remoção ---
    print("\n--- Verificando duplicados na coluna 'Prontuário' limpa ---")

    # O método .value_counts() conta quantas vezes cada valor aparece
    contagem_prontuarios = df_pacientes_limpo['Prontuário'].value_counts()

    # Filtramos a contagem para encontrar apenas os prontuários que aparecem mais de uma vez
    prontuarios_duplicados = contagem_prontuarios[contagem_prontuarios > 1]

    # Verifica se a série de duplicados está vazia ou não
    if prontuarios_duplicados.empty:
        print("✅ SUCESSO! Nenhum valor duplicado foi encontrado na coluna 'Prontuário' após a limpeza.")
    else:
        print(f"❌ ATENÇÃO! Foram encontrados {len(prontuarios_duplicados)} prontuários duplicados após a limpeza.")
        print("Abaixo estão os prontuários e a quantidade de vezes que eles aparecem:")
        # A função display formata a saída de forma mais elegante no notebook
        display(prontuarios_duplicados)

    print("-" * 50)

else:
    print("Erro: O DataFrame 'df_pacientes' não foi encontrado. Por favor, execute a célula que o carrega primeiro.")

In [ ]:
# --- 1. Verificação dos DataFrames (Opcional, mas recomendado) ---
# Garante que os DataFrames df_consolidado e df_pacientes_limpo existem antes de prosseguir.
if 'df_consolidado' in locals() and 'df_pacientes_limpo' in locals():

    # --- 2. Junção dos DataFrames (Merge) ---
    # Usamos a função pd.merge para unir os dois DataFrames.
    # how='left': Mantém todas as linhas de 'df_consolidado' (o DataFrame da esquerda).
    # left_on='prontuario_cisam': Coluna chave do DataFrame da esquerda.
    # right_on='Prontuário': Coluna chave do DataFrame da direita.
    df_completo = pd.merge(df_consolidado, df_pacientes_limpo, how='inner', left_on='prontuario_cisam', right_on='Prontuário')

    # --- 3. Exibição do Resultado ---
    print("DataFrames unidos com sucesso!")
    print("Visualizando as 5 primeiras linhas do DataFrame resultante:")
    # O display é mais adequado para notebooks, pois formata a tabela de forma mais legível.
    display(df_completo.head())

    print("\nInformações sobre o DataFrame completo:")
    df_completo.info()

    # Exibe a quantidade de valores nulos para verificar prontuários que não encontraram correspondência
    print("\nContagem de valores nulos na coluna 'Prontuário' (do df_pacientes_limpo):")
    print(f"Isto indica quantos prontuários de 'df_consolidado' não foram encontrados em 'df_pacientes_limpo'.")
    print(df_completo['Prontuário'].isnull().sum())

else:
    print("Erro: Certifique-se de que as células anteriores que criam os DataFrames 'df_consolidado' e 'df_pacientes_limpo' foram executadas com sucesso.")

In [ ]:
# --- 1. Verificação do DataFrame ---
# Garante que o DataFrame 'df_completo' existe no ambiente do notebook.
if 'df_completo' in locals():

    print(f"Número de linhas ANTES da limpeza: {len(df_completo)}")

    # --- 2. Limpeza dos Dados ---
    # A maneira mais robusta de remover linhas com valores ausentes é usando o método .dropna().
    # Este método remove, por padrão, linhas que contêm valores nulos (NaN - Not a Number).
    # O parâmetro 'subset' especifica que devemos olhar apenas para a coluna 'prontuario_cisam'
    # para decidir se uma linha deve ser removida.
    df_completo_limpo = df_completo.dropna(subset=['prontuario_cisam'])

    # --- Verificação Adicional ---
    # Às vezes, "vazio" pode significar uma string vazia ('') em vez de um valor nulo (NaN).
    # O código abaixo garante que linhas com strings vazias (ou apenas com espaços) também sejam removidas.
    # Primeiro, garantimos que a coluna é do tipo string para usar métodos de string (.str)
    df_completo_limpo['prontuario_cisam'] = df_completo_limpo['prontuario_cisam'].astype(str)
    # Em seguida, mantemos apenas as linhas onde a coluna, sem espaços, não é uma string vazia.
    df_completo_limpo = df_completo_limpo[df_completo_limpo['prontuario_cisam'].str.strip() != '']


    # --- 3. Exibição do Resultado ---
    print(f"Número de linhas APÓS a limpeza: {len(df_completo_limpo)}")

    print("\nVisualizando as primeiras linhas do DataFrame após a remoção:")
    display(df_completo_limpo.head())

    # Opcional: Se você quiser que a variável original 'df_completo' aponte para
    # o dataframe limpo, descomente a linha abaixo.
    # df_completo = df_completo_limpo.copy()
    # print("\nO DataFrame 'df_completo' foi atualizado.")

else:
    print("Erro: O DataFrame 'df_completo' não foi encontrado. Certifique-se de que a célula anterior (que une os dataframes) foi executada.")

In [ ]:
# Supondo que o DataFrame 'df_completo_limpo' já esteja em memória.

# 1. Obter a contagem de cada valor único
contagem_condicao_saude = df_completo_limpo['condicao_saude'].value_counts()

# 2. Converter a Series resultante em um DataFrame para melhor manipulação
df_contagem = contagem_condicao_saude.to_frame()

# 3. Transformar o índice (que contém os nomes das condições) em uma coluna
df_contagem = df_contagem.reset_index()

# 4. Renomear as colunas para nomes mais claros e descritivos
df_contagem.columns = ['Condicao', 'Frequencia']

# 5. Definir o caminho e o nome do arquivo de saída
caminho_saida = '../data/interim/'
nome_arquivo = 'analise_frequencia_condicao_saude.csv'
caminho_completo = os.path.join(caminho_saida, nome_arquivo)

# 6. Garantir que o diretório de destino exista
os.makedirs(caminho_saida, exist_ok=True)

# 7. Salvar o DataFrame em um arquivo CSV
# O argumento index=False evita que o índice do DataFrame (0, 1, 2...) seja salvo no arquivo
df_contagem.to_csv(caminho_completo, index=False, encoding='utf-8')

print(f"Análise de frequência da coluna 'condicao_saude' concluída.")
print(f"Os resultados foram salvos com sucesso em: '{caminho_completo}'")

# Opcional: Exibir as primeiras linhas do DataFrame gerado
print("\nAmostra dos dados salvos:")
print(df_contagem.head())

In [ ]:
# Supondo que o DataFrame 'df_completo_limpo' já esteja em memória e pronto.

# 1. Definir o caminho de saída e o nome do arquivo
caminho_saida = '../data/processed/'
nome_arquivo = 'dados_pacientes_histeroscopia_limpo.csv'
caminho_completo = os.path.join(caminho_saida, nome_arquivo)

# 2. Garantir que o diretório de destino exista
os.makedirs(caminho_saida, exist_ok=True)

# 3. Salvar o DataFrame em um arquivo CSV
#   - index=False: Evita que o índice numérico do pandas seja salvo como uma coluna no CSV.
#   - encoding='utf-8': Garante a compatibilidade e a correta gravação de caracteres especiais (acentos, etc.).
df_completo_limpo.to_csv(caminho_completo, index=False, encoding='utf-8')

print("DataFrame 'df_completo_limpo' salvo com sucesso!")
print(f"O arquivo está localizado em: '{caminho_completo}'")

In [ ]:
# URL do endpoint da API do Ollama para geração de texto
url = "http://localhost:11434/api/generate"

# Dados da requisição
data = {
    "model": "phi3:mini",
    "prompt": "Por que o céu é azul?",
    "stream": False  # Definir como False para receber a resposta completa de uma vez
}

try:
    # Fazendo a requisição POST para a API
    response = requests.post(url, json=data)
    
    # Verificando se a requisição foi bem-sucedida
    response.raise_for_status()

    # Convertendo a resposta de texto JSON para um dicionário Python
    response_data = response.json()

    # Imprimindo a resposta do modelo
    print("\nResposta do Phi3-mini:")
    print(response_data['response'])

except requests.exceptions.RequestException as e:
    print(f"Ocorreu um erro ao tentar se conectar com o Ollama: {e}")
    print("Verifique se o Ollama está em execução na porta 11434.")

In [ ]:
# Carrega a planilha para um DataFrame do pandas
try:
    df = pd.read_csv('../data/interim/criterios_encaminhamento_hd.csv')

    # Cria um dicionário vazio para armazenar os mapeamentos
    criterios_respostas = {}

    # Itera sobre as linhas do DataFrame
    for index, row in df.iterrows():
        criterio = row['Fator Específico a ser Avaliado']
        opcoes = row['Opções Categóricas para o Modelo de IA']
        
        # Verifica se os valores não são nulos
        if pd.notna(criterio) and pd.notna(opcoes):
            # Divide as opções de resposta e remove espaços em branco
            respostas = [resposta.strip() for resposta in opcoes.split('/')]
            
            # Adiciona a opção "Não informado" a lista de respostas
            respostas.append('Não informado')
            
            criterios_respostas[criterio] = respostas

    # Adiciona o novo critério manualmente ao final do dicionário
    criterios_respostas['Paciente encaminha para hd'] = ['sim', 'não', 'Não informado']

    # Imprime o dicionário resultante em formato JSON
    import json
    print(json.dumps(criterios_respostas, indent=4, ensure_ascii=False))

except FileNotFoundError:
    print("Erro: O arquivo 'criterios_encaminhamento_hd.csv' não foi encontrado.")
    print("Por favor, certifique-se de que o arquivo está no mesmo diretório que o seu notebook Jupyter.")

In [ ]:
try:
    # Carrega o arquivo CSV em um DataFrame
    df = pd.read_csv('../data/processed/dados_pacientes.csv')

    # Define as colunas que serão concatenadas
    colunas_texto = ['condicao_saude', 'motivo_consulta', 'Evolução']
    
    # Preenche os valores ausentes (NaN) com uma string vazia ''
    for col in colunas_texto:
        df[col] = df[col].fillna('')

    # Concatena as três colunas em uma nova coluna chamada 'texto_completo'
    df['texto_completo'] = df['condicao_saude'] + ' | ' + df['motivo_consulta'] + ' | ' + df['Evolução']

    # Remove as colunas de texto originais
    df = df.drop(columns=colunas_texto)
    
    # Define o nome do novo arquivo
    novo_arquivo_csv = '../data/processed/dados_pacientes_concatenado.csv'

    # Salva o DataFrame modificado em um novo arquivo CSV
    # O argumento index=False evita que o pandas salve o índice do DataFrame como uma coluna
    df.to_csv(novo_arquivo_csv, index=False, encoding='utf-8-sig')

    print(f"Arquivo '{novo_arquivo_csv}' gerado com sucesso!")
    print("\nVisualização das primeiras linhas do novo arquivo:")
    print(df.head())
    
except FileNotFoundError:
    print("Erro: O arquivo 'dados_pacientes.csv' não foi encontrado.")
    print("Por favor, certifique-se de que o arquivo está no mesmo diretório que o seu notebook Jupyter.")
except KeyError as e:
    print(f"Erro: Uma das colunas necessárias não foi encontrada no arquivo. Verifique o nome da coluna: {e}")

In [ ]:
# ==============================================================================
# ETAPA 2: CARREGAR OS DADOS DOS PACIENTES
# ==============================================================================
try:
    df_pacientes = pd.read_csv('../data/processed/dados_pacientes_concatenado.csv')
    print(f"\nArquivo 'dados_pacientes_concatenado.csv' carregado. Total de {len(df_pacientes)} pacientes para processar.")
except FileNotFoundError:
    print("ERRO CRÍTICO: O arquivo 'dados_pacientes_concatenado.csv' não foi encontrado.")
    df_pacientes = None
except Exception as e:
    print(f"ERRO ao carregar os dados dos pacientes: {e}")
    df_pacientes = None

# ==============================================================================
# ETAPA 3: FUNÇÃO PARA INTERAGIR COM O OLLAMA (PHI3-MINI)
# ==============================================================================

def extrair_informacoes_com_phi3(texto_paciente, criterios):
    """
    Envia o texto do paciente e os critérios para o Ollama e retorna as respostas.
    """
    # URL da API do Ollama que roda localmente
    url_ollama = 'http://localhost:11434/api/generate'

    # Construindo o "prompt" (a instrução) para o modelo de linguagem
    # Esta é a parte mais importante, pois ensina o modelo como se comportar.
    prompt_completo = f"""
    Você é um assistente de análise de dados de saúde.
    Sua tarefa é analisar o prontuário de um paciente e responder a uma lista de perguntas.
    Para cada pergunta, você DEVE escolher UMA das opções fornecidas.
    Baseie-se ESTRITAMENTE no texto fornecido. Se a informação não estiver no texto, escolha 'Não informado'.

    **Texto do Prontuário do Paciente:**
    "{texto_paciente}"

    **Perguntas e Opções (responda em formato JSON):**
    """
    
    # Adiciona cada critério ao prompt
    perguntas_json = {}
    for criterio, opcoes in criterios.items():
        perguntas_json[criterio] = f"Escolha uma: {opcoes}"
    
    prompt_completo += json.dumps(perguntas_json, indent=2, ensure_ascii=False)
    
    prompt_completo += "\n\n**Responda APENAS com o objeto JSON contendo as chaves (critérios) e os valores (respostas escolhidas).**"

    # Dados a serem enviados para a API do Ollama
    data = {
        "model": "phi3:mini",  # Modelo especificado
        "prompt": prompt_completo,
        "stream": False,  # Queremos a resposta completa de uma vez
        "options": {
            "temperature": 0.0 # Baixa temperatura para respostas mais diretas e menos criativas
        }
    }

    try:
        response = requests.post(url_ollama, json=data)
        response.raise_for_status()  # Lança um erro se a requisição falhar
        
        # O Ollama retorna um JSON, e a resposta do modelo está na chave "response"
        resposta_texto = response.json().get('response', '{}')
        
        # Tenta extrair o JSON da resposta do modelo
        try:
            # O modelo pode retornar o JSON dentro de ```json ... ```, vamos limpar isso.
            if '```json' in resposta_texto:
                resposta_texto = resposta_texto.split('```json\n')[1].split('```')[0]
            
            return json.loads(resposta_texto)
        except (json.JSONDecodeError, IndexError) as e:
            print(f"\n   -> Aviso: Não foi possível decodificar o JSON da resposta do modelo. Erro: {e}")
            print(f"   -> Resposta recebida: {resposta_texto}")
            return None # Retorna None se o JSON for inválido

    except requests.exceptions.RequestException as e:
        print(f"\nERRO DE CONEXÃO: Não foi possível conectar ao Ollama. Verifique se ele está rodando. Detalhes: {e}")
        return "ERRO_CONEXAO"

# ==============================================================================
# ETAPA 4: ITERAR, PROCESSAR E SALVAR OS RESULTADOS
# ==============================================================================
if criterios_respostas is not None and df_pacientes is not None:
    
    # Cria novas colunas no DataFrame para cada critério, inicializadas com 'Pendente'
    for criterio in criterios_respostas.keys():
        df_pacientes[criterio] = 'Pendente'

    print("\nIniciando o processamento dos pacientes com o modelo Phi3-mini...")
    print("-" * 70)

    # Itera por cada linha (paciente) do DataFrame
    for index, row in df_pacientes.iterrows():
        texto = row['texto_completo']
        
        print(f"Processando paciente {index + 1} de {len(df_pacientes)}...")
        
        # Chama a função que interage com o Ollama
        respostas_modelo = extrair_informacoes_com_phi3(texto, criterios_respostas)
        
        # Se a conexão com o Ollama falhar, interrompe o script
        if respostas_modelo == "ERRO_CONEXAO":
            print("Execução interrompida devido a erro de conexão.")
            break
        
        # Se o modelo retornou respostas válidas, atualiza o DataFrame
        if respostas_modelo:
            for criterio, resposta in respostas_modelo.items():
                if criterio in df_pacientes.columns:
                    # df.at é uma forma eficiente de atribuir um valor a uma célula específica
                    df_pacientes.at[index, criterio] = resposta
        else:
            print(f"   -> Falha ao obter respostas para o paciente {index + 1}.")

        # Pausa para não sobrecarregar o servidor e para podermos ler o log
        time.sleep(1) 

    print("-" * 70)
    print("Processamento concluído!")

    # Salva o resultado final em um novo arquivo CSV
    arquivo_final = 'dados_pacientes_analisados.csv'
    df_pacientes.to_csv(arquivo_final, index=False, encoding='utf-8-sig')
    
    print(f"\nArquivo '{arquivo_final}' salvo com sucesso!")
    print("\nVisualização das 5 primeiras linhas do resultado final:")
    print(df_pacientes.head())

In [ ]:
import ollama

# A lista 'messages' irá armazenar o histórico da conversa
messages = [
    {
        'role': 'user',
        'content': 'Olá! Eu quero que você atue como um tradutor de inglês para português do Brasil.',
    },
    {
        'role': 'assistant',
        'content': 'Entendido! Pode me enviar as frases em inglês que eu as traduzirei para o português brasileiro.',
    }
]

def conversar(prompt_usuario):
    """Adiciona a nova mensagem do usuário e obtém a resposta do modelo."""
    
    # Adiciona a nova mensagem ao histórico
    messages.append({
        'role': 'user',
        'content': prompt_usuario,
    })

    # Chama a API de chat, enviando todo o histórico
    response = ollama.chat(
        model='phi3:mini',
        messages=messages
    )

    # Adiciona a resposta do modelo ao histórico
    resposta_assistente = response['message']['content']
    messages.append({
        'role': 'assistant',
        'content': resposta_assistente,
    })
    
    return resposta_assistente

# Iniciando a conversa
print("Assistente: Entendido! Pode me enviar as frases em inglês que eu as traduzirei para o português brasileiro.\n")

# Primeira interação
prompt1 = "The quick brown fox jumps over the lazy dog."
print(f"Você: {prompt1}")
resposta1 = conversar(prompt1)
print(f"Assistente: {resposta1}\n")

# Segunda interação (o modelo se lembrará do contexto)
prompt2 = "How about this one: 'Never gonna give you up, never gonna let you down.'"
print(f"Você: {prompt2}")
resposta2 = conversar(prompt2)
print(f"Assistente: {resposta2}\n")

# Você pode inspecionar o histórico completo se quiser
# print("\n--- Histórico da Conversa ---")
# print(messages)

In [ ]:
# Dados a serem armazenados no arquivo JSON
dados_json = {
    "Padrão do Sangramento Uterino": [
        "Ciclos Regulares",
        "Ciclos Irregulares",
        "Ausência de Menstruação (Amenorreia)",
        "Não informado"
    ],
    "Volume do Fluxo Menstrual": [
        "Normal",
        "Aumentado (troca de absorvente a cada 1-2h)",
        "Diminuído",
        "Não informado"
    ],
    "Sangramento Pós-Menopausa": [
        "Sim",
        "Não",
        "Não informado"
    ],
    "Sangramento Fora do Período": [
        "Presente",
        "Ausente",
        "Não informado"
    ],
    "Dificuldade para Engravidar (Infertilidade)": [
        "Sim (tentando há >1 ano)",
        "Sim (tentando há <1 ano)",
        "Não se aplica",
        "Não informado"
    ],
    "Histórico de Abortos de Repetição": [
        "Nenhum",
        "1 aborto",
        "2 ou mais abortos",
        "Não informado"
    ],
    "Dor Pélvica Crônica ou Cólicas Severas": [
        "Presente",
        "Ausente",
        "Não informado"
    ],
    "Uso de Tamoxifeno": [
        "Sim",
        "Não",
        "Não informado"
    ],
    "Terapia de Reposição Hormonal (TRH)": [
        "Sim",
        "Não",
        "Não informado"
    ],
    "Histórico Familiar Relevante": [
        "Presente",
        "Ausente",
        "Desconhecido",
        "Não informado"
    ],
    "Sangramento Ativo no Exame Especular": [
        "Presente",
        "Ausente",
        "Não informado"
    ],
    "Aumento do Volume Uterino na Palpação": [
        "Sim (aumentado)",
        "Não (tamanho normal)",
        "Não informado"
    ],
    "Dificuldade em Localizar o Fio do DIU": [
        "Sim (fio não visível)",
        "Não se aplica",
        "Não informado"
    ],
    "Paciente encaminha para hd": [
        "sim",
        "não",
        "Não informado"
    ]
}

# Define o caminho e o nome do arquivo
caminho_arquivo = "../data/interim/criterios_avaliacao.json"

# Extrai o diretório do caminho do arquivo
diretorio = os.path.dirname(caminho_arquivo)

# Cria o diretório se ele não existir
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Escreve o dicionário no arquivo JSON com formatação e codificação UTF-8
with open(caminho_arquivo, 'w', encoding='utf-8') as f:
    json.dump(dados_json, f, ensure_ascii=False, indent=4)

# Imprime uma mensagem de confirmação
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

In [ ]:
# 2. Leitura do JSON
caminho_json = '../data/interim/criterios_avaliacao.json'
with open(caminho_json, 'r', encoding='utf-8') as f:
    criterios_objeto = json.load(f)

# 3. Definir os caminhos dos arquivos de entrada e saída
caminho_entrada = '../data/processed/dados_pacientes_concatenado.csv'
caminho_saida = '../data/processed/resultado_final_v2.csv'

# 4. Bloco principal de execução com tratamento de erros
try:
    # Ler o arquivo CSV para um DataFrame do pandas
    print(f"Lendo o arquivo de entrada: {caminho_entrada}")
    df = pd.read_csv(caminho_entrada)
    print("Arquivo lido com sucesso!")
    print(f"Shape original do DataFrame: {df.shape}")
    print("\nPrimeiras 5 linhas do DataFrame original:")
    print(df.head())

    # Obter a lista de novas colunas a partir das chaves do JSON
    novas_colunas = list(criterio['nome_coluna'] for criterio in criterios_objeto)
    print(f"\nAdicionando {len(novas_colunas)} novas colunas vazias...")

    # Adicionar cada chave do JSON como uma nova coluna no DataFrame
    # O valor atribuído será NaN (Not a Number), que é o padrão do pandas para dados ausentes.
    for coluna in novas_colunas:
        df[coluna] = ""

    print("Novas colunas adicionadas com sucesso!")
    print(f"Novo shape do DataFrame: {df.shape}")
    print("\nPrimeiras 5 linhas do DataFrame com as novas colunas:")
    print(df.head())

    # Salvar o DataFrame modificado em um novo arquivo CSV
    # O argumento index=False evita que o pandas salve o índice do DataFrame como uma coluna no CSV
    print(f"\nSalvando o resultado em: {caminho_saida}")
    df.to_csv(caminho_saida, index=False)
    print("Arquivo 'resultado_final.csv' salvo com sucesso!")

except FileNotFoundError:
    print(f"ERRO: O arquivo de entrada não foi encontrado no caminho especificado: {caminho_entrada}")
except Exception as e:
    print(f"Ocorreu um erro inesperado durante o processamento: {e}")

In [2]:
# Define os caminhos para os arquivos de dados.
caminho_csv = '../data/processed/resultado_final_v2.csv'
caminho_resultado_final_processado = '../data/processed/resultado_final_processado_v2.csv'
caminho_json = '../data/interim/criterios_avaliacao.json'

colunas_para_tratar_como_string = {
    'padrao_sangramento_uterino': 'str',
    'volume_fluxo_menstrual': 'str',
    'sangramento_pos_menopausa': 'str',
    'sangramento_fora_periodo': 'str',
    'dificuldade_engravidar': 'str',
    'historico_aborto': 'str',
    'dor_pelvica': 'str',
    'tamoxifeno': 'str',
    'terapia_reposição_hormonal_trh': 'str',
    'historico_familiar': 'str',
    'sangramento_ativo_exame_specular': 'str',
    'aumento_volume_uterino_palpacao': 'str',
    'localizar_fio_diu': 'str',
    'encaminhada_hd': 'str'
}

print('-' * 50)
print('OP ID: 001')
print('Início da execução do algoritmo')
print('-' * 50)
print('\n\n')


# Carrega o arquivo JSON. 
# A estrutura esperada é um objeto onde cada chave tem uma lista de strings como valor.
# Ex: {"chave1": ["opcaoA", "opcaoB"], "chave2": ["opcaoC", "opcaoD"]}
with open(caminho_json, 'r', encoding='utf-8') as f:
    criterios_objeto = json.load(f)

# Carrega o DataFrame a partir do CSV.
df_pacientes = pd.read_csv(caminho_csv, dtype=colunas_para_tratar_como_string)

# Armazena a quantidade de voltas do loop no dataframe. Tem utilizade para a depuração do código
index_loop = 0

# Itera sobre cada paciente no DataFrame.
for linha_paciente in df_pacientes.itertuples():
    index_loop += 1
    if hasattr(linha_paciente, 'texto_completo'):
        
        texto_do_paciente = str(linha_paciente.texto_completo)

        print('-' * 50)
        print('OP ID: 002')
        print(f'Iniciando execução para paciente {index_loop}')
        print('-' * 50)
        print('\n\n')

        # Itera sobre as chaves do objeto JSON (os critérios).
        for criterio in criterios_objeto:
            
            # --- ALTERAÇÃO: Acessa a lista de opções de resposta para o critério atual. ---
            opcoes_de_resposta = criterio['alternativas']
            
            # Formata a lista de opções em uma string clara para o prompt.
            # Ex: "'Opção 1', 'Opção 2', 'Opção 3'"
            opcoes_formatadas = ", ".join([f"{chr(65+i)}) {opcao}" for i, opcao in enumerate(opcoes_de_resposta)])

            # Monta o prompt com a nova instrução.

            prompt_atual = (
                f"PRONTUÁRIO DA PACIENTE:\n{texto_do_paciente}\n\n"
                f"PERGUNTA:\n {criterio['pergunta']}\n"
                f"{opcoes_formatadas}\n\n"
                "RESPOSTA:"
            )

            print('-' * 50)
            print('OP ID: 003')
            print(f'Mensagem a ser enviada para o modelo: ')
            print(prompt_atual)
            print('-' * 50)
            print('\n\n')

            response = ollama.generate(
                model='phi3:mini',
                prompt=prompt_atual,
                options={
                    'num_predict': 1
                }
            )

            if 'response' in response:
                resposta_texto = response['response']
                tempo_para_resposta = response['total_duration']
                
                # Salva a resposta do modelo na coluna correspondente do DataFrame.
                df_pacientes.loc[linha_paciente.Index, criterio['nome_coluna']] = resposta_texto

                print('-' * 50)
                print('OP ID: 004')
                print(f'Resposta do modelo: {resposta_texto}')
                print(f'Tempo para a resposta {tempo_para_resposta / 1e9:.2f}')
                print('-' * 50)
                print('\n\n')
            else:
                print('-' * 50)
                print('OP ID: 004')
                print(f'Houve um erro na resposta do modelo para essa iteração {index_loop}')
                print('-' * 50)
                print('\n\n')
                df_pacientes.loc[linha_paciente.Index, criterio] = "ERRO NA RESPOSTA DO MODELO"
    else:
        print('-' * 50)
        print('OP ID: 002')
        print(f'ERRO: atributo "texto_completo" não encontrado. Iteração pulada no index {index_loop}')
        print('-' * 50)
        print('\n\n')
    print('-' * 50)
    print('OP ID: 005')
    print(f'Paciente {index_loop} totalmente analisado')
    print('-' * 50)
    print('\n\n')
# Salva o DataFrame modificado de volta ao arquivo CSV.
df_pacientes.to_csv(caminho_resultado_final_processado, index=False, encoding='utf-8')

print(f"Processamento concluído. O DataFrame foi atualizado e salvo em '{caminho_resultado_final_processado}'.")

--------------------------------------------------
OP ID: 001
Início da execução do algoritmo
--------------------------------------------------



--------------------------------------------------
OP ID: 002
Iniciando execução para paciente 1
--------------------------------------------------



--------------------------------------------------
OP ID: 003
Mensagem a ser enviada para o modelo: 
PRONTUÁRIO DA PACIENTE:
Ainda tenho ciclo menstrual ativo (ainda menstruo regularmente) | Dores no baixo ventre polipos | Preciso fazer exame pra tira os polipos | Lauren , 52 anos, dum : há 9 meses ( perimenopausa) .

tem diagnostico de polipo desde 2020 .

procura teleconsulta para realização de marcação de histeroscopia diagnostica.

usg 13/1/23 -> EE 1 CM, colposcopia evidenciando polipo endocervical.



cd: marco histeroscopia diagnostica. 

PERGUNTA:
 Qual é o padrão de sangramento da paciente?
A) Ciclos Regulares, B) Ciclos Irregulares, C) Ausência de Menstruação (Amenorreia), D) Não in

/tmp/ipykernel_41602/85216575.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_pacientes.loc[linha_paciente.Index, criterio['nome_coluna']] = resposta_texto


--------------------------------------------------
OP ID: 004
Resposta do modelo: C
Tempo para a resposta 3.05
--------------------------------------------------



--------------------------------------------------
OP ID: 003
Mensagem a ser enviada para o modelo: 
PRONTUÁRIO DA PACIENTE:
Ainda tenho ciclo menstrual ativo (ainda menstruo regularmente) | Dores no baixo ventre polipos | Preciso fazer exame pra tira os polipos | Lauren , 52 anos, dum : há 9 meses ( perimenopausa) .

tem diagnostico de polipo desde 2020 .

procura teleconsulta para realização de marcação de histeroscopia diagnostica.

usg 13/1/23 -> EE 1 CM, colposcopia evidenciando polipo endocervical.



cd: marco histeroscopia diagnostica. 

PERGUNTA:
 A paciente apresenta aumento do volume uterino na palpação?
A) Sim, B) Não, C) Não informado

RESPOSTA:
--------------------------------------------------



--------------------------------------------------
OP ID: 004
Resposta do modelo: C
Tempo para a resposta 3.10
---